In [1]:
!pip install textblob

In [2]:
from __future__ import unicode_literals, print_function
import textblob
import pandas as pd
#import sys
#reload(sys)
#sys.setdefaultencoding("utf8")

In [3]:
textblob.TextBlob("Hello World I hate you").sentiment.polarity

-0.8

In [4]:
textblob.TextBlob("Hello World I love you").sentiment.polarity

0.5

In [5]:
textfile = "./hackpadtext.txt"

In [6]:
rawdata = pd.read_csv(textfile, header=None, sep="\n", encoding="utf-8")
#print rawdata

In [7]:
polarities = []

In [10]:
for i in rawdata.index:
    data = rawdata.loc[i].values[0]
    #print(data)
    polarity = textblob.TextBlob(data).sentiment.polarity
    print(polarity)
    polarities.append(polarity)

0.625
0.0193181818182
0.15
0.0
0.191666666667
0.25
0.1
0.291666666667
0.375
0.0
0.0
-0.0888888888889
0.0752525252525
0.08875
0.375
0.0
-0.316666666667
0.1
0.166419294991
0.055
0.5
0.0
0.0
0.0
-0.0218181818182
-0.225952380952
0.0
0.0
0.0718614718615
0.5
0.375
0.216666666667
0.0985347985348
0.175
0.25
0.4375
0.625
0.113541666667
0.0
-0.0218181818182
0.308444444444
0.45
0.2625
0.205
0.25
0.180952380952
0.259523809524
0.116666666667
0.0
-0.129166666667
0.4625
0.6875
0.459090909091
0.0
0.0764583333333
0.147619047619
0.375
0.104761904762
0.25
0.35
0.25
-0.0111772486772
0.5
0.239930555556
0.2
0.35
0.375
0.0625
0.0
0.0
0.131818181818
-0.148863636364
0.105555555556
0.1125
0.09375
0.0
0.21875
0.15
0.333333333333
-0.0107142857143
0.4
0.0
0.5
0.240374331551
0.625
0.7
0.0
0.875
0.177232142857
-0.11
0.5
0.175
-0.09375
0.0416666666667
0.0
0.125793650794
0.0
0.0113636363636
0.8
0.0
0.386666666667
0.169047619048
0.386666666667
0.15


In [11]:
average_feels = sum(polarities)/len(polarities)
print(average_feels)

0.186349310623
